# Internals of Generators


@steph.bsky.social | @0xkasteph

kasteph.eth.limo

github.com/kasteph

<img src="rc.png" width="50"/>
<img src="pl.png" width="250"/>


## Agenda

- Iterables
- Iterators
- Generators
- Implementation of Generators in CPython

## Iterable

An object that implements `__iter__` or `__getitem__`. Let's see some examples.

In [1]:
for token in ["pycon", "apac", "2023"]:
    print(token)

pycon
apac
2023


In [2]:
for token in ("are", "tuples", "iterable", "?"):
    print(token)

are
tuples
iterable
?


In [3]:
for c in "Yes!":
    print(c.upper())
    

Y
E
S
!


In [4]:
wallet = {"eur": 10, "yen": 3000}
for key, values in wallet.items():
    print(key)

eur
yen


An `Iterable` returns an `Iterator` object.

## Iterator

> An iteration interface that objects can provide to control the behaviour of `for` loops. 

> ~ [PEP-0234](https://peps.python.org/pep-0234/) by Ka-Ping Yee and Guido van Rossum

An object that:
- implements `__next__`
- raises `StopIteration`

Let's create our own iterator. 👷‍♀️🛠️

In [5]:
from typing import Any, Optional


class Node:
    def __init__(self, data: Any, next_: Optional["Node"] = None):
        self.data = data
        self.next = next_
        
    def __next__(self) -> "Node":
        if not self.next:
            raise StopIteration("The value for next is `None`.")
        return self.next

In [6]:
root = Node("a", next_=Node("b", next_=Node("c")))
current_node = root
while current_node:
    print(current_node.data)
    current_node = current_node.next

a
b
c


In [7]:
class Node:
    def __init__(self, data: Any, next_: Optional["Node"] = None):
        self.data = data
        self.next = next_
        
    def __next__(self) -> "Node":
        if not self.next:
            raise StopIteration("The value for next is `None`.")
        return self.next

    def __iter__(self) -> "Node":
        current_node = self
        while current_node:
            yield current_node
            current_node = current_node.next

In [8]:
root = Node("a", 
            next_=Node("b", 
                       next_=Node("c")))

In [9]:
for node in root:
    print(node.data)

a
b
c


In [10]:
type(root.__iter__())

generator

## `yield`

```python
def __iter__(self) -> "Node":
    current_node = self
    while current_node:
        yield current_node
        current_node = current_node.next
```

Using `yield` turns `__iter__` into a generator function. 

## Generators

Introduced in [PEP-0255](https://peps.python.org/pep-0255/).

Motivating example: `cpython/Lib/tokenize.py` on the [`v2.1` tag](https://github.com/python/cpython/blob/2.1/Lib/tokenize.py#L108-L112):

```python
def tokenize(readline, tokeneater=printtoken):
    try:
        tokenize_loop(readline, tokeneater)
    except StopTokenizing:
        pass
```

### Remembering State

```python
def tokenize(readline, tokeneater=printtoken):
    ...
```
Users often have to remember the state between callbacks.

For e.g. [`tabnanny.py`](https://github.com/python/cpython/blob/2.1/Lib/tabnanny.py#L80C5-L82) has a state machine in global variables.

### Eager Evaluation

In PEP-0255, the authors suggested an alternative to "produce an entire parse of the Python program at once, in a large list" <sup>[source](https://peps.python.org/pep-0255/#motivation)</sup>. This solves the state management issue.

Can you imagine why this would be problematic?

1. Large inputs may not fit in memory because the memory allocation cannot be precomputed.
2. The whole output may not even be necessary.

### A Simple Generator Function

In [11]:
def fruit_box():
    yield "banana"
    yield "mango"
    yield "kiwi"

In [12]:
type(fruit_box), type(fruit_box())

(function, generator)

In [13]:
for fruit in fruit_box():
    print(fruit)

banana
mango
kiwi


In [14]:
def veggie_box():
    yield "onion"
    yield "spinach"
    

def grocery_list():
    yield from fruit_box()
    yield from veggie_box()
    
    
def grocery_list2():
    for fruit in fruit_box():
        yield fruit
    for veggie in veggie_box():
        yield veggie

In [15]:
list(grocery_list()), list(grocery_list2())

(['banana', 'mango', 'kiwi', 'onion', 'spinach'],
 ['banana', 'mango', 'kiwi', 'onion', 'spinach'])

## Implementation of Generators in CPython

The simplest recipe for generators 🥘:

1. Some kind of structure.
2. Something that creates that structure.
3. Something that executes the generator.

### Structure

```C
typedef struct {
    /* The gi_ prefix is intended to remind of generator-iterator. */
    _PyGenObject_HEAD(gi)
} PyGenObject;
```
[`genobject.h`](https://github.com/python/cpython/blob/main/Include/cpython/genobject.h#L34)


```C
#define _PyGenObject_HEAD(prefix)
    PyObject_HEAD
    PyObject *prefix##_weakreflist;
    PyObject *prefix##_name;
    PyObject *prefix##_qualname;
    _PyErr_StackItem prefix##_exc_state;
    PyObject *prefix##_origin_or_finalizer;
    char prefix##_hooks_inited;
    char prefix##_closed;
    char prefix##_running_async;
    int8_t prefix##_frame_state;
    PyObject *prefix##_iframe[1];
```
[genobject.h](https://github.com/python/cpython/blob/main/Include/cpython/genobject.h#L14-L29)

Given the struct `PyGenObject` and macro `_PyGenObject_HEAD`, we can expand it to:

```C
typedef struct {
    PyObject *gi_weakreflist;    
    PyObject *gi_name;    
    PyObject *gi_qualname;

    _PyErr_StackItem gi_exc_state;
    PyObject *gi_origin_or_finalizer;
    char gi_hooks_inited;  

    char gi_closed;
    char gi_running_async;

    int8_t gi_frame_state;
    PyObject *gi_iframe[1];

} PyGenObject;
```

A structure that holds the generator's code object, its current state, and the frame it is executing in.

### Initalizer

```C
PyObject *
PyGen_New(PyFrameObject *f)
{
    return gen_new_with_qualname(&PyGen_Type, f, NULL, NULL);
}
```
[genobject.c](https://github.com/python/cpython/blob/main/Objects/genobject.c#L1007-L1011)

```C
PyGen_New(PyFrameObject *f)
```

🤔 `PyFrameObject` ???

### `PyFrameObject`

Used by the Python interpreter to execute a function.

- Local and global variables
- Bytecode
- Execution stack

[docs](https://docs.python.org/3/c-api/frame.html)

![](./generator_frame.png)
[pythontutor.org permalink](https://pythontutor.com/render.html#code=def%20fruit_box%28%29%3A%0A%20%20%20%20yield%20%22banana%22%0A%20%20%20%20yield%20%22mango%22%0A%20%20%20%20yield%20%22strawberry%22%0A%0Afor%20fruit%20in%20fruit_box%28%29%3A%0A%20%20%20%20print%28fruit%29&cumulative=false&curInstr=9&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=311&rawInputLstJSON=%5B%5D&textReferences=false)

```C
static PyObject *
gen_new_with_qualname(PyTypeObject *type, PyFrameObject *f,
                      PyObject *name, PyObject *qualname)
{
    PyCodeObject *code = _PyFrame_GetCode(f->f_frame);
    int size = code->co_nlocalsplus + code->co_stacksize; 👈 1
    PyGenObject *gen = PyObject_GC_NewVar(PyGenObject, type, size); 
    👆2
    ...
    /* Copy the frame */ 
    ... 👇 3
    _PyInterpreterFrame *frame = (_PyInterpreterFrame *)gen->gi_iframe;
    _PyFrame_Copy((_PyInterpreterFrame *)f->_f_frame_data, frame);
    ...
    _PyObject_GC_TRACK(gen); 👈 4
    return (PyObject *)gen;
}
```
1. Calculate the size needed.
2. Allocate the memory.
3. Copy the frame to preserve state and have an independent execution context from the caller.
4. Track the object for the GC.

`gen_new_with_qualname` calculates the size of the generator and allocates the memory for the generator to be created.

### Execution

py: `__next__()` → C: `gen_iternext()` → C: `gen_send_ex2()`

`gen_send_ex2` [source code](https://github.com/python/cpython/blob/main/Objects/genobject.c#L169-L264).

> 💡`gen_send_ex` is an abstraction over `gen_send_ex2` and is used for implementing the `send()`, `close()`, `throw()` methods of generators.

```C
...
gen->gi_frame_state = FRAME_EXECUTING; 👈 1
EVAL_CALL_STAT_INC(EVAL_CALL_GENERATOR);
result = _PyEval_EvalFrame(tstate, frame, exc); 👈✨ 2
assert(tstate->exc_info == prev_exc_info); 👈 3
assert(gen->gi_exc_state.previous_item == NULL);
assert(gen->gi_frame_state != FRAME_EXECUTING);
assert(frame->previous == NULL);
...
```
1. Set the state to executing.
2. Run the generator code until `yield`, `return`, `Exception`.
3. All following assertions: `assert(python.thread_state == generator.state)`

```C
if (result) {
    if (gen->gi_frame_state == FRAME_SUSPENDED) {
        *presult = result; 👈 1
        return PYGEN_NEXT; 👈 2
    }
    assert(result == Py_None || !PyAsyncGen_CheckExact(gen));
    if (result == Py_None && !PyAsyncGen_CheckExact(gen) && !arg) {
        Py_CLEAR(result); 👈 3
    }
}
else {  
    // make sure it's not a StopIteration
}

assert(gen->gi_exc_state.exc_value == NULL);
assert(gen->gi_frame_state == FRAME_CLEARED);
*presult = result;
return result ? PYGEN_RETURN : PYGEN_ERROR;  👈 4
```
1. Store the result if it's yielded (ie, generator paused and to be resumed later).
2. Notify caller that there's a next value.
3. Clear the result because generator is done.
4. Let the caller know if we return a value, `PYGEN_RETURN`, or raise an exception, `PYGEN_ERROR`.

😵‍💫 What just happened?

We just took a look at how generators are implemented under the hood.

But how does all of this C code back up the claim that "generators use memory
efficiently?"

1. The frame object in `PyGenObject` maintains the state of the generator. Only the current value and state is saved in memory instead of all intermediate results.
2. `PyObject_GC_NewVar` which was used in `gen_new_with_qualname` to allocate memory.
3. Because generators participate in garbage collection, that memory is reclaimed as soon as it's no longer needed.

## Thanks! 🙏

And here's some resources:

- [PEP-0234](https://peps.python.org/pep-0234/)
- [PEP-0255](https://peps.python.org/pep-0255/)
- [The Python Wiki: Iterator](https://wiki.python.org/moin/Iterator)
- [Standard Types: Iterator Types](https://docs.python.org/3/library/stdtypes.html#iterator-types)
- [Expressions: `yield`](https://docs.python.org/3/reference/expressions.html#yieldexpr)
- [Expressions: Generator-iterator methods](https://docs.python.org/3/reference/expressions.html#generator-iterator-methods)
- [GitHub: python/cpython](https://github.com/python/cpython)